In [37]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
import sys
!{sys.executable} -m pip install -U pandas-profiling catboost ipywidgets
!jupyter nbextension enable --py widgetsnbextension

In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import pandas_profiling
from pandas_profiling.utils.cache import cache_file

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, plot_confusion_matrix
from sklearn.ensemble import RandomForestClassifier

In [69]:
data = pd.read_csv('train.csv')

In [70]:
data['MSSubClass']=data['MSSubClass'].astype('object')
data['YearBuilt']=data['YearBuilt'].astype('object')
data['YearRemodAdd']=data['YearRemodAdd'].astype('object')
data['MoSold']=data['MoSold'].astype('object')
data['YrSold']=data['YrSold'].astype('object')
data['MasVnrArea']=data['MasVnrArea'].astype('float64')
data['GarageYrBlt']=data['GarageYrBlt'].astype(pd.Int64Dtype())
data['GarageYrBlt']=data['GarageYrBlt'].astype('object')

data['Alley'] = data['Alley'].fillna('NoAlley')
data['FireplaceQu'] = data['FireplaceQu'].fillna('NoFireplace')
data['PoolQC'] = data['PoolQC'].fillna('NoPool')
data['Fence'] = data['Fence'].fillna('NoFence')
data['MiscFeature'] = data['MiscFeature'].fillna('NoMiscFeature')
data['GarageCond'] = data['GarageCond'].fillna('NoGarage')
data['GarageQual'] = data['GarageQual'].fillna('NoGarage')
data['GarageType'] = data['GarageType'].fillna('NoGarage')
data['GarageFinish'] = data['GarageFinish'].fillna('NoGarage')
data['GarageYrBlt'] = data['GarageYrBlt'].fillna('NoGarage')
data['BsmtFinSF2'] = data['BsmtFinSF2'].fillna('NoBasement')
data['BsmtFinSF1'] = data['BsmtFinSF1'].fillna('NoBasement')
data['BsmtFinType1'] = data['BsmtFinType1'].fillna('NoBasement')
data['BsmtFinType2'] = data['BsmtFinType2'].fillna('NoBasement')
data['BsmtExposure'] = data['BsmtExposure'].fillna('NoBasement')
data['BsmtQual'] = data['BsmtQual'].fillna('NoBasement')
data['BsmtCond'] = data['BsmtCond'].fillna('NoBasement')
data['MasVnrType'] = data['MasVnrType'].fillna('NoMasVnr')
data['MasVnrArea'] = data['MasVnrArea'].fillna(0)

data['LotFrontage']=data['LotFrontage'].fillna(data['LotFrontage'].median())

data[['LotFrontage','BsmtUnfSF','TotalBsmtSF','GarageCars','GarageArea','BsmtFullBath','BsmtHalfBath' ]]= data[['LotFrontage','BsmtUnfSF','TotalBsmtSF','GarageCars','GarageArea','BsmtFullBath','BsmtHalfBath']].fillna(data[['LotFrontage','BsmtUnfSF','TotalBsmtSF','GarageCars','GarageArea','BsmtFullBath','BsmtHalfBath']].median())

for column in ['MSZoning','Electrical', 'Utilities','Exterior1st','Exterior2nd','KitchenQual', 'Functional','SaleType']:
    data[column].fillna(data[column].mode()[0], inplace=True)
    data[column].fillna(data[column].mode()[0], inplace=True)

In [71]:
data[:1]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NoAlley,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NoFireplace,Attchd,2003,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NoPool,NoFence,NoMiscFeature,0,2,2008,WD,Normal,208500


In [72]:
pd.set_option('display.max_columns', None)

In [73]:
y = data['SalePrice']
X = data.drop(columns = ['SalePrice', 'Id'])
## Perform the split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [74]:
from catboost import CatBoostRegressor, Pool, metrics, cv
from sklearn.metrics import roc_auc_score

In [75]:
cat_features = np.where(X.dtypes != float)[0]

In [83]:
params = {
    'iterations': 500,
    'learning_rate': 0.1,
    'eval_metric': 'RMSE',
    'random_seed': 42,
    'od_type': 'Iter',
    'od_wait': 40,
    'logging_level': 'Silent',
}

In [84]:
pool = Pool(data=X, label=y, cat_features=cat_features)

In [85]:
train_pool = Pool(X_train, y_train, cat_features=cat_features)
validate_pool = Pool(X_test, y_test, cat_features=cat_features)

In [86]:
model = CatBoostRegressor(**params).fit(pool, plot=True)

In [87]:
cv_params = model.get_params()
cv_params.update({
    'loss_function': metrics.RMSE()
})
cv_data = cv(
    pool,
    cv_params,
    plot=True
)

In [88]:
cv_data

,iterations,test-RMSE-mean,test-RMSE-std,train-RMSE-mean,train-RMSE-std
0,0,181178.360112,5592.126457,180997.096643,2403.794850
1,1,165574.393302,5869.256559,165546.744644,2062.739607
2,2,151771.533165,5808.923889,151622.636673,2047.454430
3,3,139470.373068,5646.638578,139099.538479,2289.939442
4,4,128314.431901,5689.352896,127748.392921,2164.613636
...,...,...,...,...,...
495,495,31237.888754,6622.846893,10730.306536,633.880831
496,496,31239.807810,6619.735637,10727.301090,633.726161
497,497,31238.665884,6621.586951,10726.020876,633.673254
498,498,31238.646898,6621.617733,10725.747783,633.662971


In [173]:
data = pd.read_csv('test.csv')

In [174]:
data['MSSubClass']=data['MSSubClass'].astype('object')
data['YearBuilt']=data['YearBuilt'].astype('object')
data['YearRemodAdd']=data['YearRemodAdd'].astype('object')
data['MoSold']=data['MoSold'].astype('object')
data['YrSold']=data['YrSold'].astype('object')
data['MasVnrArea']=data['MasVnrArea'].astype('float64')
data['GarageYrBlt']=data['GarageYrBlt'].astype(pd.Int64Dtype())
data['GarageYrBlt']=data['GarageYrBlt'].astype('object')
data['BsmtFinSF1']=data['BsmtFinSF1'].astype(pd.Int64Dtype())
data['BsmtFinSF1']=data['BsmtFinSF1'].astype('object')

data['BsmtFinSF2']=data['BsmtFinSF2'].astype(pd.Int64Dtype())
data['BsmtFinSF2']=data['BsmtFinSF2'].astype('object')

data['BsmtUnfSF']=data['BsmtUnfSF'].astype(pd.Int64Dtype())
data['TotalBsmtSF']=data['TotalBsmtSF'].astype(pd.Int64Dtype())
data['BsmtFullBath']=data['BsmtFullBath'].astype(pd.Int64Dtype())
data['BsmtHalfBath']=data['BsmtHalfBath'].astype(pd.Int64Dtype())
data['GarageCars']=data['GarageCars'].astype(pd.Int64Dtype())
data['GarageArea']=data['GarageArea'].astype(pd.Int64Dtype())

data['Alley'] = data['Alley'].fillna('NoAlley')
data['FireplaceQu'] = data['FireplaceQu'].fillna('NoFireplace')
data['PoolQC'] = data['PoolQC'].fillna('NoPool')
data['Fence'] = data['Fence'].fillna('NoFence')
data['MiscFeature'] = data['MiscFeature'].fillna('NoMiscFeature')
data['GarageCond'] = data['GarageCond'].fillna('NoGarage')
data['GarageQual'] = data['GarageQual'].fillna('NoGarage')
data['GarageType'] = data['GarageType'].fillna('NoGarage')
data['GarageFinish'] = data['GarageFinish'].fillna('NoGarage')
data['GarageYrBlt'] = data['GarageYrBlt'].fillna('NoGarage')
data['BsmtFinSF2'] = data['BsmtFinSF2'].fillna('NoBasement')
data['BsmtFinSF1'] = data['BsmtFinSF1'].fillna('NoBasement')
data['BsmtFinType1'] = data['BsmtFinType1'].fillna('NoBasement')
data['BsmtFinType2'] = data['BsmtFinType2'].fillna('NoBasement')
data['BsmtExposure'] = data['BsmtExposure'].fillna('NoBasement')
data['BsmtQual'] = data['BsmtQual'].fillna('NoBasement')
data['BsmtCond'] = data['BsmtCond'].fillna('NoBasement')
data['MasVnrType'] = data['MasVnrType'].fillna('NoMasVnr')
data['MasVnrArea'] = data['MasVnrArea'].fillna(0)

data['LotFrontage']=data['LotFrontage'].fillna(data['LotFrontage'].median())

data[['LotFrontage','BsmtUnfSF','TotalBsmtSF','GarageCars','GarageArea','BsmtFullBath','BsmtHalfBath' ]]= data[['LotFrontage','BsmtUnfSF','TotalBsmtSF','GarageCars','GarageArea','BsmtFullBath','BsmtHalfBath']].fillna(data[['LotFrontage','BsmtUnfSF','TotalBsmtSF','GarageCars','GarageArea','BsmtFullBath','BsmtHalfBath']].median())

for column in ['MSZoning','Electrical', 'Utilities','Exterior1st','Exterior2nd','KitchenQual', 'Functional','SaleType']:
    data[column].fillna(data[column].mode()[0], inplace=True)
    data[column].fillna(data[column].mode()[0], inplace=True)

In [175]:
data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NoAlley,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468,LwQ,144,270,882,GasA,TA,Y,SBrkr,896,0,0,896,0,0,1,0,2,1,TA,5,Typ,0,NoFireplace,Attchd,1961,Unf,1,730,TA,TA,Y,140,0,0,0,120,0,NoPool,MnPrv,NoMiscFeature,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NoAlley,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923,Unf,0,406,1329,GasA,TA,Y,SBrkr,1329,0,0,1329,0,0,1,1,3,1,Gd,6,Typ,0,NoFireplace,Attchd,1958,Unf,1,312,TA,TA,Y,393,36,0,0,0,0,NoPool,NoFence,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NoAlley,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791,Unf,0,137,928,GasA,Gd,Y,SBrkr,928,701,0,1629,0,0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997,Fin,2,482,TA,TA,Y,212,34,0,0,0,0,NoPool,MnPrv,NoMiscFeature,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NoAlley,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602,Unf,0,324,926,GasA,Ex,Y,SBrkr,926,678,0,1604,0,0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998,Fin,2,470,TA,TA,Y,360,36,0,0,0,0,NoPool,NoFence,NoMiscFeature,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NoAlley,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263,Unf,0,1017,1280,GasA,Ex,Y,SBrkr,1280,0,0,1280,0,0,2,0,2,1,Gd,5,Typ,0,NoFireplace,Attchd,1992,RFn,2,506,TA,TA,Y,0,82,0,0,144,0,NoPool,NoFence,NoMiscFeature,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NoAlley,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,Twnhs,2Story,4,7,1970,1970,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Unf,0,Unf,0,546,546,GasA,Gd,Y,SBrkr,546,546,0,1092,0,0,1,1,3,1,TA,5,Typ,0,NoFireplace,NoGarage,NoGarage,NoGarage,0,0,NoGarage,NoGarage,Y,0,0,0,0,0,0,NoPool,NoFence,NoMiscFeature,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NoAlley,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,TwnhsE,2Story,4,5,1970,1970,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,252,Unf,0,294,546,GasA,TA,Y,SBrkr,546,546,0,1092,0,0,1,1,3,1,TA,6,Typ,0,NoFireplace,CarPort,1970,Unf,1,286,TA,TA,Y,0,24,0,0,0,0,NoPool,NoFence,NoMiscFeature,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NoAlley,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1Story,5,7,1960,1996,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,ALQ,1224,Unf,0,0,1224,GasA,Ex,Y,SBrkr,1224,0,0,1224,1,0,1,0,4,1,TA,7,Typ,1,TA,Detchd,1960,Unf,2,576,TA,TA,Y,474,0,0,0,0,0,NoPool,NoFence,NoMiscFeature,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NoAlley,Reg

In [176]:
data.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [177]:
ids  = data["Id"]
data = data.drop(columns = ['Id'])

In [178]:
predictions = model.predict(data)

In [179]:
predictions

array([118549.58533704, 145148.99624333, 167442.72902613, ...,
       154143.53423319, 115851.46094233, 204204.75443121])

In [180]:
predict_data['Id' ] = ids 

In [181]:
predict_data['SalePrice' ] = predictions 

In [182]:
predict_data.to_csv('catboost_final.csv',index=False)